In [1]:
import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *

import pandas as pd

p = Project.objects.get(pk=260)
qs = p.query_set.all()
q = Query.objects.get(pk=8843)

dids = set(Doc.objects.filter(query__in=qs).values_list('pk', flat=True))
len(dids)

565085

In [2]:
regenerate_docinfo = False
if regenerate_docinfo:
    df = pd.DataFrame.from_dict(Doc.objects.filter(pk__in=dids).values('id','PY','title','wosarticle__dt'))
    df.columns = ["id","Publication Year","Title","Keywords"]
    df.head()
    df.to_csv('data/0c_doc_info.csv',index=False)

In [3]:

django.db.connection.close()

dos = DocOwnership.objects.filter(tag__query__project=p, relevant__gt=0)

doc_ids = set(dos.values_list('doc__id',flat=True))

# Get relevant / not relevant decisions
do_df = pd.DataFrame.from_dict(
    dos.values('doc__id','doc__title','doc__content','user__username','relevant')
)
do_df.loc[do_df['relevant']==0,'val'] = np.NaN
do_df.loc[do_df['relevant']==1,'val'] = 1
do_df.loc[do_df['relevant']==2,'val'] = 0
do_df.loc[do_df['relevant']==3,'val'] = np.NaN
do_df.head()

# Get category assignment
cats = Category.objects.filter(project=p).exclude(name__icontains="hidden>")
df = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        category__in=cats,
        doc__in=doc_ids
    ).values(
        'doc__id',
        'doc__title',
        'doc__content',
        'category__name',
        'category__level',
        'user__username',
        'countries__name',
        'texts__name',
        'selection_tier'
    )
)
# Filter so that it matches the relevant set
df = df[
    (df[['doc__id','user__username']].apply(tuple, axis=1).isin(do_df[['doc__id','user__username']].apply(tuple, axis=1))) | 
    (df['user__username']=="Auto")
]

# Rename category to capture level
df['category__level'] = df['category__level'].astype(str)
df['Category Name'] = df[['category__level','category__name']].apply(lambda x: ' - '.join(x), axis=1)
#df['cat'] = df[['category__level','category__name','user__username']].apply(lambda x: ' - '.join(x), axis=1)

# Capture the primary/secondary value (1 for primary, 2 for secondary)
df['val'] = df['selection_tier']

# Set the value to the country / text where appropriate and j
df.loc[~pd.isna(df['countries__name']),'val'] = df.loc[~pd.isna(df['countries__name']),'countries__name']
df.loc[~pd.isna(df['texts__name']),'val'] = df.loc[~pd.isna(df['texts__name']),'texts__name']

cats = df['Category Name'].unique()

df.head()

,doc__id,doc__title,doc__content,category__name,category__level,user__username,countries__name,texts__name,selection_tier,Category Name,val
0,3212104,Carbon emission quota allocating on coal and e...,This paper considers the fair and efficient pr...,1. Economic instruments,4,vasudevan@mcc-berlin.net,None,None,1,4 - 1. Economic instruments,1
1,222862,A COMPREHENSIVE SOLUTION FOR DEVELOPMENT OF LA...,To combat climate change and fulfill the natio...,1. Economic instruments,4,galm,None,None,1,4 - 1. Economic instruments,1
2,1894344,Algerian Renewable Energy Projects for Clean E...,"Renewable energy sources RES (solar, wind reso...",1. Economic instruments,4,doebbeling@mcc-berlin.net,None,None,2,4 - 1. Economic instruments,2
3,3775854,China's Carbon Pricing Based on Heterogeneous ...,"To address climate change, the carbon emission...",1. Economic instruments,4,edmondson@hertie-school.org,None,None,1,4 - 1. Economic instruments,1
4,51374,THE CHOICE BETWEEN AUCTIONING AND GRANDFATHERI...,Based on Hillman's (1982) political support fu...,1. Economic instruments,4,vasudevan@mcc-berlin.net,None,None,1,4 - 1. Economic instruments,1


In [4]:
DocOwnership.objects.filter(tag__query__project=p,doc=3556).values('tag__title','tag__id','user__username')

<QuerySet [{'tag__title': 'Probably relevant random sample 9', 'tag__id': 6540, 'user__username': 'vasudevan@mcc-berlin.net'}, {'tag__title': 'Probably relevant random sample 9', 'tag__id': 6540, 'user__username': 'lucy.banisch@icloud.com'}]>

In [5]:
# Merge relevant/not relevant with categories
do_df['Category Name'] = "0 - relevant"

merged_df = pd.concat([
    df[['doc__id','doc__title','doc__content','user__username','Category Name','val']],
    do_df[['doc__id','doc__title','doc__content','user__username','Category Name','val']]
])
merged_df.tail()

,doc__id,doc__title,doc__content,user__username,Category Name,val
5630,1838661,Cultural and Social Elements in the Developmen...,In the context of climate change and energy cr...,doebbeling@mcc-berlin.net,0 - relevant,0
5631,315555,"GLOBAL EXPERIMENTALIST GOVERNANCE, INTERNATION...",Global experimentalist governance has emerged ...,galm,0 - relevant,1
5632,1799426,"Underdetermination, Model-ensembles and Surpri...",As climate policy decisions are decisions unde...,galm,0 - relevant,1
5633,3827028,Establishment a recycled green building materi...,This study modifies a dynamic computable gener...,vasudevan@mcc-berlin.net,0 - relevant,1
5634,3827028,Establishment a recycled green building materi...,This study modifies a dynamic computable gener...,galm,0 - relevant,1


In [6]:
# fill tag values with 0s where they have been rated
dudf_wide = (merged_df[['doc__id','doc__title','doc__content','user__username','Category Name','val']]
             .pivot_table(index=['doc__id','doc__title','doc__content','user__username'],columns="Category Name", values="val", aggfunc=np.sum)
             #.fillna(0)
            ).reset_index()

cats = [x for x in cats if x in dudf_wide.columns]

dudf_wide[cats] = dudf_wide[cats].fillna(0)

policy_columns=[x for x in dudf_wide.columns if re.match("^3",x)]
include_policy_columns = [x for x in policy_columns if "not policy related" not in x.lower() and "broader" not in x.lower()]
print(policy_columns)
print(include_policy_columns)
post_policy_columns=[x for x in dudf_wide.columns if re.match("^[4-9]",x) or re.match("^[0-9]{2}",x)]
#dudf_wide.loc[dudf_wide['3 - 0. Not policy related']==1,post_policy_columns] = np.NaN
dudf_wide.loc[dudf_wide[policy_columns].sum(axis=1,min_count=1)==0,post_policy_columns] = np.NaN
dudf_wide.loc[dudf_wide[include_policy_columns].sum(axis=1,min_count=1)>0,"INCLUDE"] = 1
dudf_wide.loc[dudf_wide[include_policy_columns].sum(axis=1,min_count=1)==0,"INCLUDE"] = 0

dudf_wide.to_csv('data/0_all_labels.csv',index=False)

dudf_wide.tail()

['3 - 0. Not policy related', '3 - 1. Analysis of an existing policy / concrete proposal', '3 - 2. Analysis of a generic policy', '3 - 3. Broader governance and policy']
['3 - 1. Analysis of an existing policy / concrete proposal', '3 - 2. Analysis of a generic policy']


Category Name,doc__id,doc__title,doc__content,user__username,0 - relevant,10 - 3. Quantitative,10 - 4. Qualitative,11 - countries,12 - policy name,2 - 0. Mitigation,...,8 - 10. Coastal systems and low-lying areas,8 - 11. Ocean systems,8 - 12. Food,"8 - 13. Human settlements, industry and infrastructure","8 - 14. Human health, well-being and security",8 - 15. Cross-sectoral,9 - 0. Ex-post,9 - 1. Ex-ante,9 - 2. Ex-ante (IAM),INCLUDE
5530,3878364,Occupants' Awareness of and Satisfaction with ...,With growing awareness of the need to reduce g...,vasudevan@mcc-berlin.net,1,0.0,0.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5531,3878440,Urban consumers' willingness to pay for higher...,Rapid urbanization poses accelerating pressure...,ceposadap@gmail.com,1,1.0,0.0,China,Energy Efficiency Labeling System (EELS),1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5532,3878440,Urban consumers' willingness to pay for higher...,Rapid urbanization poses accelerating pressure...,edmondson@hertie-school.org,1,0.0,0.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5533,3878516,Life cycle greenhouse gas impacts of a connect...,"As technological advancements progress, the au...",ceposadap@gmail.com,1,0.0,0.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5534,3878516,Life cycle greenhouse gas impacts of a connect...,"As technological advancements progress, the au...",vasudevan@mcc-berlin.net,1,0.0,0.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Now get a single value for each document by choosing the most common selection
import statistics
def resolve_conflicts(x):
    valid_codes = [y for y in x if pd.notna(y)]
    l = len(set(valid_codes))
    if l==0: # If no valid codes return NA
        return np.NaN
    elif l==1: # If there is only one unique code, return the first
        return valid_codes[0]
    else: # Otherwise find the majority decision
        rm = np.mean(rounded_codes)
        if rm > 0.5: # overall, a majority coded positive (primary or secondary)
            return 0.66
        elif rm==0.5: # even split, can't decide
            return np.NaN
        else: # mostly negative
            return 0
    
# First make it long, so that each row is a doc-category-user rating
xdf = dudf_wide.melt(id_vars=['doc__id','doc__title','doc__content','user__username'])

# Remove categories 11...+ as they are strings not int and need to be dealt with separately
xdf = xdf[~xdf['Category Name'].isin(['11 - countries','12 - policy name'])]

xdf['value'] = xdf['value'].replace(2,0.5)

# Now group by doc and category, and resolve conflicts
xdf = xdf.groupby(['doc__id','doc__title','doc__content','Category Name'])['value'].agg(lambda x: resolve_conflicts(x)).reset_index()

# weight samples according to the level of conflict and the value
# Unanimous 1s get a weight of 1
xdf.loc[xdf['value'].isin([0,1]),"sample_weight"] = 1
# Majority 1s, or mixture of 1s and 2s get a weight of 0.66
xdf.loc[xdf['value']==0.66,"sample_weight"] = 0.66
# Unanimous 2s get a weight of 0.5
xdf.loc[xdf['value']==0.5,"sample_weight"] = 0.5

# recode back to integer values
xdf['value'] = xdf['value'].replace(0.66,1)
xdf['value'] = xdf['value'].replace(0.5,1)


# Now pivot so that each category is a column
seen_df = xdf.drop(columns="sample_weight").pivot(index=["doc__id","doc__title","doc__content"],columns="Category Name").reset_index()
# xdf["Category Name"] = xdf["Category Name"] + "_sample_weight"
# weights_df = xdf.drop(columns="value").pivot(index=["doc__id","doc__title","doc__content"],columns="Category Name").reset_index()
print(seen_df.shape)

seen_df.columns = [col[1] if col[1] !="" else col[0] for col in seen_df.columns]

print(seen_df[pd.isna(seen_df['0 - relevant'])].shape)

seen_df.head()

cols_4 = [x for x in seen_df.columns if re.match("^4 -",x)]
seen_df[seen_df['doc__id']==1828017][["doc__id"]+cols_4]

(2580, 103)
(0, 103)


,doc__id,4 - 1. Economic instruments,4 - 2. Regulatory Instruments,"4 - 3. Information, education and training","4 - 4. Governance, strategies and targets",4 - 5. Agreements
1777,1828017,0.0,0.0,0.0,1.0,0.0


In [9]:
cols_3 = [x for x in seen_df.columns if re.match("^3 -",x)]
seen_df[seen_df['doc__id']==1824928][["doc__id"]+cols_3 + cols_4]

,doc__id,3 - 0. Not policy related,3 - 1. Analysis of an existing policy / concrete proposal,3 - 2. Analysis of a generic policy,3 - 3. Broader governance and policy,4 - 1. Economic instruments,4 - 2. Regulatory Instruments,"4 - 3. Information, education and training","4 - 4. Governance, strategies and targets",4 - 5. Agreements
1767,1824928,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# First make it long, so that each row is a doc-category-user rating
xdf = dudf_wide.melt(id_vars=['doc__id','doc__title','doc__content','user__username'])

# Remove categories 11...+ as they are strings not int and need to be dealt with separately
xdf = xdf[(xdf['Category Name'].str.match("^3{1}")) & (xdf['doc__id']==1824928)]
xdf

,doc__id,doc__title,doc__content,user__username,Category Name,value
42509,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,edmondson@hertie-school.org,3 - 0. Not policy related,1
42510,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,vasudevan@mcc-berlin.net,3 - 0. Not policy related,1
48044,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,edmondson@hertie-school.org,3 - 1. Analysis of an existing policy / concre...,0
48045,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,vasudevan@mcc-berlin.net,3 - 1. Analysis of an existing policy / concre...,0
53579,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,edmondson@hertie-school.org,3 - 2. Analysis of a generic policy,0
53580,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,vasudevan@mcc-berlin.net,3 - 2. Analysis of a generic policy,0
59114,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,edmondson@hertie-school.org,3 - 3. Broader governance and policy,0
59115,1824928,Forest fires and air pollution: A local and a ...,A numerical system developed to simulate the e...,vasudevan@mcc-berlin.net,3 - 3. Broader governance and policy,0


In [11]:
renamings = [
    ("5 - 2.07. Spatial and land-use planning","5 - 2.09. Spatial and land-use planning"),
    ("5 - 2.07. Spatial and land-use planning","6 - 2.09. Spatial and land-use planning"),
    ("5 - 5.17. Inter/transnational agreements","5 - 5.20. Inter/transnational agreements"),
    ("5 - 5.17. Inter/transnational agreements","6 - 5.20. Inter/transnational agreements"),
    ("Analysis of a specific policy","Analysis of an existing policy / concrete proposal"),
    ("Governance and planning", "Governance, strategies and targets"),
    ("5 - 2.08. Other regulation", "5 - 2.10. Other regulation"),
    ("5 - 2.08. Other regulation", "6 - 2.10. Other regulation"),
    ("Other institutions or agencies", "Other institutional arrangements"),
    ("5 - 3.11. Behavioural interventions", "5 - 3.13. Behavioural interventions"),
    ("5 - 3.11. Behavioural interventions", "6 - 3.02. Behavioural interventions"),
    ("5 - 3.12. Education and training","5 - 3.14. Education and training"),  
    
]

In [12]:
# Loop through the comparison/resolution files and resolve labelled dataset
resolutions = pd.DataFrame()

for fname in os.listdir("decision_resolutions"):
    if ".xlsx" in fname:
        try:
            cdf = pd.read_excel(f'decision_resolutions/{fname}',sheet_name="comparison")
        except:
            print(f"Cannot read {fname}")
    dcolumn = [x for x in cdf.columns if "decision" in x.lower()][0]
    cdf = cdf.rename(columns={dcolumn: "decision"})
    cdf = cdf[pd.notna(cdf["decision"])]
    cdf = cdf[cdf['Category Name'].str.match("^[0-9]{1}(?![1-2])")]
    
    cdf['fname'] = fname
    
    resolutions = pd.concat([resolutions, cdf])
    
    for i, row in cdf.iterrows():
        try:
            x = int(row["decision"])
        except:
            print(f"could not parse {row['decision']}")
            continue
        cname = row['Category Name']
        
        for r1, r2 in renamings:
            cname = cname.replace(r1,r2)
        if cname not in seen_df.columns:
            matches = [x for x in seen_df.columns if cname.split('. ')[1] in x]
            if len(matches) == 1:
                cname = matches[0]
            else:
                seen_df[cname] = 0
                continue
            
        seen_df.loc[(seen_df['doc__id']==row['doc__id']),cname] = row["decision"]
        
print(resolutions.shape)
resolutions.to_csv('data/0_conflict_resolutions.csv',index=False)

could not parse generic, strategies
could not parse New Category
could not parse Demonstration 
could not parse expert advice 
could not parse ?
could not parse infrastructure investment + new category 
could not parse NEW CATEGORY
could not parse NEW CATEGORY
could not parse NEW CATEGORY
could not parse NEW CATEGORY
could not parse Tradable Quotas
could not parse new category
could not parse carbon payments
Cannot read .~lock.Probably relevant random sample 17_Shraddha_Lucy.xlsx#
(7702, 21)


In [13]:
resolutions[resolutions['doc__id']==1865162]

,doc__id,doc__title,doc__content,Category Name,ceposadap@gmail.com,edmondson@hertie-school.org,galm,lucy.banisch@icloud.com,agreement,decision,...,doebbeling@mcc-berlin.net,vasudevan@mcc-berlin.net,muef,Spalte1,doebbeling@posteo.net,flac,lamw,Unnamed: 10,Unnamed: 8,Unnamed: 9
11842,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,3 - 1. Analysis of an existing policy / concre...,0,NaN,NaN,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11843,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,3 - 2. Analysis of a generic policy,2,NaN,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11845,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,4 - 1. Economic instruments,0,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11846,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,4 - 2. Regulatory Instruments,0,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11847,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,"4 - 3. Information, education and training",0,NaN,NaN,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11848,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,"4 - 4. Governance, strategies and targets",0,NaN,NaN,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11849,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,4 - 5. Agreements,2,NaN,NaN,2,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11861,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,5 - 4.16. Planning,0,NaN,NaN,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11883,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,6 - 4.01. Targets & goals,0,NaN,NaN,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11888,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,Recently many events occurred in the energy sc...,7 - 0. Supranational and international,1,NaN,NaN,2,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
corrections = pd.read_csv('decision_resolutions/master/corrections.csv', quotechar="'")

for i, row in corrections.iterrows():
    try:
        x = int(row[dcolumn])
    except:
        print(f"could not parse {row[dcolumn]}")
        continue
    cname = row['Category Name']

    for r1, r2 in renamings:
        cname = cname.replace(r1,r2)
    if cname not in seen_df.columns:
        matches = [x for x in seen_df.columns if cname.split('. ')[1] in x]
        if len(matches) == 1:
            cname = matches[0]
        else:
            print(cname)
            continue

    seen_df.loc[(seen_df['doc__id']==row['doc__id']),cname] = row[dcolumn]


9 - 0. Ex-post - prediction


In [15]:
corrections[corrections['doc__id']==1824928]

,doc__id,Category Name,decision
4,1824928,3 - 1. Analysis of an existing policy / concre...,2


In [16]:
# Now label Inclusion (simply the presence of any policy type)
include_policy_columns = [x for x in policy_columns if "not policy related" not in x.lower() and "3 - 3." not in x.lower()]

seen_df.loc[seen_df[include_policy_columns].max(axis=1).isin([1,2,np.NaN]),"INCLUDE"] = 1
seen_df.loc[seen_df[include_policy_columns].max(axis=1)==0,"INCLUDE"] = 0
seen_df.loc[seen_df["2 - 0. Mitigation"]==0,"INCLUDE"] = 0
#seen_df.loc[pd.isna(seen_df[include_policy_columns]).any(axis=1),"INCLUDE"] = 1
seen_df[["doc__id","doc__title","0 - relevant"] + policy_columns + ["INCLUDE"]].head(15)

,doc__id,doc__title,0 - relevant,3 - 0. Not policy related,3 - 1. Analysis of an existing policy / concrete proposal,3 - 2. Analysis of a generic policy,3 - 3. Broader governance and policy,INCLUDE
0,156,THE EXPECTED GREENHOUSE BENEFITS FROM DEVELOPI...,1.0,1.0,0.0,0.0,0.0,0.0
1,337,Energy forecast till 2020 for the German Gover...,1.0,1.0,0.0,0.0,0.0,0.0
2,921,Ontology engineering in provenance enablement ...,1.0,1.0,0.0,0.0,0.0,0.0
3,1448,"Planning, governance and rural futures in Aust...",1.0,0.0,0.0,0.0,0.0,0.0
4,3556,Will economic restructuring in China reduce tr...,1.0,0.0,2.0,0.0,0.0,1.0
5,3679,Forcing and response in simulated 20th and 21s...,1.0,1.0,0.0,0.0,0.0,0.0
6,3683,Egeson's (George's) transtridecadal weather cy...,0.0,0.0,0.0,0.0,0.0,0.0
7,4793,An emerging equilibrium in the EU emissions tr...,1.0,0.0,1.0,0.0,0.0,1.0
8,4929,Adapting production forests in southern Sweden...,1.0,0.0,0.0,0.0,0.0,0.0
9,5220,Dual climate change responsibility: on moral d...,1.0,1.0,0.0,1.0,0.0,1.0


In [17]:
seen_df[seen_df['doc__id']==1865162][["doc__id","doc__title","0 - relevant"] + policy_columns + cols_4 + ["INCLUDE"]].head(15)

,doc__id,doc__title,0 - relevant,3 - 0. Not policy related,3 - 1. Analysis of an existing policy / concrete proposal,3 - 2. Analysis of a generic policy,3 - 3. Broader governance and policy,4 - 1. Economic instruments,4 - 2. Regulatory Instruments,"4 - 3. Information, education and training","4 - 4. Governance, strategies and targets",4 - 5. Agreements,INCLUDE
1882,1865162,NEW ROLE FOR SENSORS IN ENERGY MANAGEMENT SYSTEMS,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0


In [18]:
# Tag whether the document is part of a representative sample
django.db.connection.close()
seen_df["representative_sample"] = 0
random_tags = Tag.objects.filter(query__project=p, title__icontains="representative")
random_ids = set(DocOwnership.objects.filter(tag__in=random_tags, relevant__gt=0).values_list('doc__id',flat=True))
seen_df.loc[seen_df['doc__id'].isin(random_ids),"representative_sample"] = 1

In [19]:
seen_df['representative_relevant'] = 1
kw_tags = Tag.objects.filter(query__project=p, title__icontains="keyword")
kw_ids = set(DocOwnership.objects.filter(tag__in=kw_tags, relevant__gt=0).values_list('doc__id',flat=True))
seen_df.loc[seen_df['doc__id'].isin(kw_ids),"representative_relevant"] = 0

In [20]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1,2)
labels = ["Non-representative sample","Representative sample"]
for name, group in seen_df.groupby(['representative_sample']):
    ax = axs[name]
    ax.set_title(labels[name])
    
    group.groupby('INCLUDE').size().plot.bar(ax=ax)

In [21]:
seen_df.rename(
    columns={"doc__id":"id","doc__content":"content","doc__title":"title"}
).to_csv('data/0_labelled_documents.csv',index=False)

In [22]:
rewrite_unseen = False

if rewrite_unseen:

    columns = [
        'id',
        'content',
        'title',
        'wosarticle__de',
        'wosarticle__wc'
    ]

    unseen_docs = Doc.objects.filter(
        id__in=dids,
        content__iregex='\w',
    ).exclude(pk__in=seen_df['doc__id'])

    unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
    unseen_df = unseen_df[columns]
    unseen_df['relevant'] = 0
    unseen_df['seen'] = 0
    unseen_df['random'] = 0

    print(unseen_df.shape)
    unseen_df.to_csv('data/0_unlabelled_documents.csv', index=False)
    unseen_df.head()